In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

import glob

from sklearn.decomposition import PCA

#from pandas_profiling import ProfileReport

# Import Exported CSV files from Previous Notebooks

In [2]:
# Read hdr_df.csv
human_development_reports_df = pd.read_csv('hdr_df.csv')

# Read country_ids.csv
country_ids = pd.read_csv('country_ids.csv')

# Read Codebook

## 3 columns to focus on: Variable, Label, Value Scheme Detailed

In [3]:
def read_codebook(file, sheet1, sheet2):
    xls = pd.ExcelFile(file)
    student = pd.read_excel(xls, sheet1)
    school = pd.read_excel(xls, sheet2)
    
    return student, school

In [4]:
stu_con_8, sch_con_8 = read_codebook('T19_G8_Codebooks/T19_G8_Codebook.xlsx', 'BCGM7', 'BSGM7')

# Read Main Data

In [5]:
BCG_files = glob.glob('T19_G8_SAS Data/BCG' + "/*.sas7bdat")

BCG_list = []

for file in BCG_files:
    BCG = pd.read_sas(file)
    BCG_list.append(BCG)

#All BCG files; total of 9,925 rows × 98 columns
BCG_merged_all = pd.concat(BCG_list, axis=0, ignore_index=True)

In [6]:
BSG_files = glob.glob('T19_G8_SAS Data/BSG' + "/*.sas7bdat")

BSG_list_455 = []
BSG_list_357 = []

for file in BSG_files:
    if len((pd.read_sas(file)).columns) == 455:
        BSG_455 = pd.read_sas(file)
        BSG_list_455.append(BSG_455)
        
    elif len((pd.read_sas(file)).columns) == 357:
        BSG_357 = pd.read_sas(file)
        BSG_list_357.append(BSG_357)

#BSG files with 455 columns; total of 219,835 rows × 455 columns
BSG_merged_455 = pd.concat(BSG_list_455, axis=0, ignore_index=True)

#BSG files with with 357 columns; total of 32,455 rows × 357 columns
BSG_merged_357 = pd.concat(BSG_list_357, axis=0, ignore_index=True)

#All BSG files; total of 252290 rows × 455 columns
BSG_merged_all = pd.concat([BSG_merged_455, BSG_merged_357], axis=0, ignore_index=True)

In [7]:
df_8 = BSG_merged_all.merge(BCG_merged_all, how='inner', on=['IDCNTRY', 'IDSCHOOL'])

## Specify columns to use for grade 8

In [8]:
#Specify columns for PCA
math_interest_cols = [col for col in df_8.columns if 'SBM16' in col]
math_perception_cols = [col for col in df_8.columns if 'SBM19' in col]
math_benefit_cols = [col for col in df_8.columns if 'SBM20' in col]
science_interest_cols = [col for col in df_8.columns if 'SBS22' in col]
science_perception_cols = [col for col in df_8.columns if 'SBS24' in col]
science_benefit_cols = [col for col in df_8.columns if 'SBS25' in col]

belong_cols = [col for col in df_8.columns if 'SBG13' in col]
bully_cols = [col for col in df_8.columns if 'SBG14' in col]
shortage_cols = [col for col in df_8.columns if 'CBG13' in col]
school_support_cols = [col for col in df_8.columns if 'CBG15' in col]
school_problem_cols = [col for col in df_8.columns if 'CBG16' in col]

teacher_support_cols = ['BCBG14A', 'BCBG14B', 'BCBG14C', 'BCBG14D']
parents_support_cols = ['BCBG14E', 'BCBG14F', 'BCBG14G', 'BCBG14H']
student_motivation_cols = ['BCBG14I', 'BCBG14J', 'BCBG14K']

In [9]:
pca_columns = math_interest_cols + math_perception_cols + math_benefit_cols + \
              science_interest_cols + science_perception_cols + science_benefit_cols + \
              belong_cols + bully_cols + shortage_cols + school_support_cols + \
              school_problem_cols + teacher_support_cols + parents_support_cols + \
              student_motivation_cols

In [10]:
id_columns = ['IDCNTRY', 'IDSCHOOL', 'IDCLASS', 'IDSTUD']

In [11]:
school_general = ['BCBG03A', 'BCBG05B', 'BCBG06A', 'BCBG06B', 'BCBG06C', 'BCBG07', 'BCBG08A', 'BCBG10B', 'BCBG12']

In [12]:
student_general = ['BSBG01', 'BSBG03', 'BSBG04', 'BSBG05A', 'BSBG05B', 'BSBG05C', 'BSBG05D', 'BSBG06A',
                   'BSBG06B', 'BSBG07', 'BSBG10', 'BSBG11A', 'BSBG11B', 'BSBM15']

In [13]:
homework_time = [col for col in df_8.columns if 'SBM42B' in col]

In [14]:
extra_lessons = [col for col in df_8.columns if 'SBM27A' in col]

In [15]:
score_columns = ['BSMMAT01', 'BSSSCI01']

In [16]:
all_columns = id_columns + student_general + school_general + pca_columns + student_general + \
              homework_time + extra_lessons + score_columns

In [17]:
df_8 = df_8[all_columns]

In [18]:
# Drop rows if any of the pca columns is with missing values
df_8 = df_8.dropna(axis=0, subset=pca_columns, how='any')

In [19]:
# Drop rows if the two main score columns is with missing values
df_8 = df_8.dropna(axis=0, subset=score_columns, how='any')

## Fix Column Values for Principal Component Analysis

In [20]:
# Change column names and map values to match with the direction of the responses of the rest
# of the values in the same group (change negative questions to positive ones, and reverse the
# values of the answers)

def map_values(col_name):
    df_8[col_name] = df_8[col_name].map({1: 4, 2: 3, 3: 2, 4:1})
    return df_8[col_name]

In [21]:
# Identify columns to fix
cols_to_fix = ['BSBM16B', 'BSBM16C', 'BSBM19B', 'BSBM19C', 'BSBM19E', 'BSBM19H',
               'BSBM19I', 'BSBS22B', 'BSBS22C', 'BSBS24B', 'BSBS24C', 'BSBS24G', 
               'BSBS24H']

In [22]:
# Fix columns
for col in cols_to_fix:
    map_values(col)

In [23]:
# Categorize columns for math and science for PCA
math_interest = df_8[math_interest_cols]
math_perception = df_8[math_perception_cols]
math_benefit = df_8[math_benefit_cols]
science_interest = df_8[science_interest_cols]
science_perception = df_8[science_perception_cols]
science_benefit = df_8[science_benefit_cols]

In [24]:
# Categorize columns for other measures for PCA
belong = df_8[belong_cols]
bully = df_8[bully_cols]
shortage = df_8[shortage_cols]
school_support = df_8[school_support_cols]
school_problem = df_8[school_problem_cols]

teacher_support = df_8[teacher_support_cols]
parents_support = df_8[parents_support_cols]
student_motivation = df_8[student_motivation_cols]

# Principal Component Analysis

In [26]:
def pca_caculation(df):
    
    #Specify columns for PCA
    X = df[df.columns.tolist()]
    
    #Scale column values
    x = StandardScaler().fit_transform(X)
    
    #PCA caculation
    pca = PCA(n_components=1)
    principal_components = pca.fit_transform(x)
    
    #Add PCA column to dataframe
    df['PCA_values'] = principal_components
    
    return df

In [27]:
#All pca dfs
pca_dfs = [math_interest, math_perception, math_benefit, science_interest, science_perception, 
           science_benefit, belong, bully, shortage, school_support, school_problem,
           teacher_support, parents_support, student_motivation]

In [28]:
#Apply function to all pca dfs
for i, df in enumerate(pca_dfs):
    df = pca_caculation(df)
    df_8[i] = df['PCA_values']

In [29]:
df_8 = df_8.drop(pca_columns, axis=1)

In [30]:
df_8 = df_8.rename(columns={0: 'math_interest', 1: 'math_perception', 2: 'math_benefit', 3: 'science_interest',
                     4: 'science_perception', 5: 'science_benefit', 6: 'belong', 7: 'bully',
                     8: 'shortage', 9: 'school_support', 10: 'school_problem', 11: 'teacher_support',
                     12: 'parents_support', 13: 'student_motivation'})

# Labels

In [31]:
# clean stu_con_8 columns to match with df_8
stu_con_8_columns = stu_con_8.iloc[2:, 0:2].set_index('Variable')
stu_8_labels = stu_con_8_columns.loc[stu_con_8_columns.index.intersection(df_8.columns)].dropna().reset_index()

In [32]:
# clean sch_con_4 columns to match with df_4
sch_con_8_columns = sch_con_8.iloc[5:, 0:2].set_index('Variable')
sch_8_labels = sch_con_8_columns.loc[sch_con_8_columns.index.intersection(df_8.columns)].dropna().reset_index()

In [33]:
#merge stu_con_4 and sch_con_4, and replace labels
sch_stu_8_labels = stu_8_labels.merge(sch_8_labels, how='outer')
sch_stu_8_labels['Label'] = sch_stu_8_labels['Label'].str.lower()

sch_stu_8_labels = sch_stu_8_labels.\
                            replace('gen\\\\', '', regex=True).\
                            replace('students background\\\\', '', regex=True).\
                            replace('disadva', 'disadvantage', regex=True).\
                            replace('sch character\\\\', '', regex=True).\
                            replace('tch', 'teacher', regex=True).\
                            replace('sch', 'school', regex=True).\
                            replace('std', 'student', regex=True).\
                            replace('often speak <lang of test> at home', 
                                    'how often student speak the language of the test at home', regex=True).\
                            replace('amount of books in your home', 'amount of books at home', regex=True).\
                            replace('home possess\\\\computer tablet', 'computer or tablet at home', regex=True).\
                            replace('home possess\\\\study desk', 'study desk at home', regex=True).\
                            replace('home possess\\\\own room', 'own room at home', regex=True).\
                            replace('home possess\\\\internet connection', 'internet connection at home', regex=True).\
                            replace('how often\\\\tired', 'how often student is tired', regex=True).\
                            replace('how often\\\\hungry', 'how often student is hungry', regex=True).\
                            replace('mat\\\\how often\\\\work on your own',
                                    'how often student works on his/her own in math', regex=True).\
                            replace('sci\\\\how often conduct experiments',
                                    'how often conduct experiments in science', regex=True).\
                            replace('mathematics', 'math', regex=True).\
                            replace('1st plausible value', '', regex=True).\
                            replace('1st pv', '', regex=True).\
                            replace('teacherrs', 'teacher', regex=True).\
                            replace('highest lvl of edu of <parent/guardian a>', 
                                    'highest level of education of 1st parent/guardian', regex=True).\
                            replace('highest lvl of edu of <parent/guardian b>', 
                                    'highest level of education of 2nd parent/guardian', regex=True).\
                            replace('how far in edu do you expect to go', 
                                    'how far in education student is expected to go', regex=True).\
                            replace('about how often absent from schoolool', 
                                    'how often student is absent from school', regex=True).\
                            replace('math\\\\work on your own', 'work on your own math', regex=True).\
                            replace('math\\\\minutes spent on hmwork\\\\mat', 
                                    'minutes spent on math homework', regex=True).\
                            replace('math\\\\extra lessons last 12 month\\\\math',
                                    'extra math lessons in last 12 months', regex=True)

In [34]:
# update new labels to df_8
new_col_naming = dict(zip(sch_stu_8_labels['index'], sch_stu_8_labels['Label']))
df_8 = df_8.rename(columns=new_col_naming)

# rename country_ids to match df_8
country_ids = country_ids.rename(columns={'country_ID':'IDCNTRY'})

In [35]:
# add country names
df_8_final = df_8.merge(country_ids)

In [36]:
# add human development reports
df_8_final = df_8_final.merge(human_development_reports_df)

In [37]:
# Export to csv
df_8.to_csv('grade_8_hdr.csv', index=False)

In [38]:
df_8_final

,IDCNTRY,IDSCHOOL,IDCLASS,IDSTUD,sex of student,how often student speak the language of the test at home,amount of books at home,computer or tablet at home,study desk at home,own room at home,...,Ratio of education and health expenditure to military expenditure,Gender Inequality Index,GII Rank,Maternal mortality ratio,Adolescent birth rate,Share of seats in parliament held by women,Share at least some secondary education Female,Share at least some secondary education Male,Labour force participation rate Female,Labour force participation rate Male
0,7842.0,5001.0,500102.0,50010202.0,2.0,2.0,2.0,1.0,1.0,1.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
1,7842.0,5001.0,500102.0,50010208.0,2.0,2.0,NaN,1.0,1.0,1.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
2,7842.0,5001.0,500102.0,50010209.0,2.0,2.0,2.0,1.0,1.0,1.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
3,7842.0,5001.0,500102.0,50010210.0,2.0,2.0,2.0,1.0,1.0,1.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
4,7842.0,5001.0,500102.0,50010211.0,2.0,2.0,1.0,1.0,2.0,NaN,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115417,784.0,5623.0,562301.0,56230124.0,2.0,1.0,1.0,2.0,2.0,2.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
115418,784.0,5623.0,562301.0,56230126.0,2.0,1.0,4.0,1.0,1.0,2.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
115419,784.0,5623.0,562301.0,56230127.0,2.0,1.0,5.0,1.0,1.0,1.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389
115420,784.0,5623.0,562301.0,56230131.0,2.0,1.0,2.0,1.0,1.0,1.0,...,NaN,0.079,18.0,3.0,6.546,50.0,76.02257,81.02851,52.391,93.389


# Factor Loadings

In [39]:
def get_factor_loadings(df):
    cols = [col for col in df.columns if col != 'PCA_values']
    df = df[cols]
    x = StandardScaler().fit_transform(df)
    pca = PCA(n_components=1)
    pca.fit(x)
    loadings = pca.components_ * np.sqrt(pca.explained_variance_)
    loadings_df = pd.DataFrame({'variable': df.columns.tolist(), 'loading': loadings[0]})
    return loadings_df

In [40]:
f_loading = get_factor_loadings(science_interest)

In [41]:
tempt_df = sch_con_8_columns.reset_index()

In [42]:
col_naming = dict(zip(tempt_df['Variable'], tempt_df['Label']))

In [43]:
f_loading['variable'] = f_loading['variable'].map(col_naming)

In [44]:
f_loading

,variable,loading
0,SCI\AGREE\ENJOY LEARNING SCIENCE,0.861074
1,SCI\AGREE\WISH HAVE NOT TO STUDY SCIENCE,0.615526
2,SCI\AGREE\SCIENCE IS BORING,0.668109
3,SCI\AGREE\LEARN INTERESTING THINGS,0.793355
4,SCI\AGREE\LIKE SCIENCE,0.901496
5,SCI\AGREE\LOOK FORWARD TO LEARN SCI,0.855421
6,SCI\AGREE\HOW THINGS WORK,0.722778
7,SCI\AGREE\LIKE SCIENCE EXPERIMENTS,0.611409
8,SCI\AGREE\FAVORITE SUBJECT,0.865509


In [46]:
# f_loading.to_csv('f_loading_8_sci_int.csv', index=False)